In [1]:
!pip install transformers peft torch datasets accelerate bitsandbytes sentence_transformers evaluate rouge nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U bitsandbytes

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `mytoken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mytoken`


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
import evaluate
from peft import PeftModel
import json
import os
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import numpy as np

In [5]:
!unzip lora_tuned_model.zip

Archive:  lora_tuned_model.zip
   creating: lora_tuned_model/
  inflating: lora_tuned_model/special_tokens_map.json  
  inflating: lora_tuned_model/tokenizer_config.json  
  inflating: lora_tuned_model/README.md  
  inflating: lora_tuned_model/tokenizer.model  
  inflating: lora_tuned_model/tokenizer.json  
  inflating: lora_tuned_model/adapter_model.safetensors  
  inflating: lora_tuned_model/adapter_config.json  


In [6]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                  torch_dtype=torch.float16,
                                                  device_map="auto",
                                                  load_in_4bit=True #comment out for non-lora?
                                                  )

model = PeftModel.from_pretrained(base_model,
                                  "./lora_tuned_model",
                                  )


embed_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
filename = "prompts_sample_n1.json"
with open(filename, "r", encoding="utf-8") as f:
  data = json.load(f)
print(f"Loaded {len(data)} prompts.")

Loaded 200 prompts.


In [8]:
pairs = []
clean_output = []
tokenizer.pad_token_id = tokenizer.eos_token_id
for index, item in enumerate(data[:200]):
    if index % 100 == 0:
        print(f"Processed {index} prompts.")

    prompt = item.get("prompt", "").strip()
    instruction = item.get("Instruction", "").strip()
    full_prompt = prompt + ("\n" + instruction if instruction else "")
    completion = item.get("ground_truth", "").strip()


    prediction = item.get("predicted_output", "").strip()


    tokenized = tokenizer(full_prompt, return_tensors="pt")
    input_ids = tokenized.input_ids.to(base_model.device)
    attn_mask = tokenized.attention_mask.to(base_model.device)
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attn_mask, max_new_tokens=50)  # Adjust max_new_tokens as needed
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(full_prompt):]
    clean_output.append(predicted_text.split('\n')[0])

    pairs.append((full_prompt, completion, predicted_text))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed 0 prompts.


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
S

Processed 100 prompts.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [9]:
with open("pairs_lora_full.json", "w") as f:
    json.dump(pairs, f)

In [10]:
for i,o in enumerate(clean_output):
  pairs[i] = (pairs[i][0], pairs[i][1], o)
with open("pairs_lora_clean_full.json", "w") as f:
    json.dump(pairs, f)

In [11]:
filename = "pairs_lora_full.json"
with open(filename, "r", encoding="utf-8") as f:
    pairs = json.load(f)

In [12]:
print(pairs[0][0])
print('===================')
print(pairs[0][1])
print('===================')
print(pairs[0][2])

[INST] Character Info: Tony Last: The novel's protagonist, a gentleman devoted to his estate, Hetton Abbey. His naive attachment to tradition and his wife, Brenda, leads to his ultimate downfall.
John Andrew: Tony and Brenda Last's young son, whose tragic death marks the turning point of the novel and accelerates Tony's misfortune.


Dialogue History: John Andrew: Is mummy coming down to-day, daddy?
Tony Last: Yes.
John Andrew: Who else?
Tony Last: Someone called Jenny Abdul Akbar.
John Andrew: What a silly name. Is she foreign?
Tony Last: I don't know.
John Andrew: Sounds foreign, doesn't she, daddy? D'you think she won't be able to talk any English? Is she black?
Tony Last: Mummy says not.
John Andrew: Oh... who else?
Tony Last: Lady Cockpurse.

Instruction: Continue the dialogue for 1 lines. 
 [/INST]
John Andrew: The monkey-woman. You know she wasn't a bit like a monkey except perhaps her face and I don't think she had a tail because I looked as close as anything... unless perhaps 

In [13]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0d432e0c020ce0bd273189821f48d76a83f8a74d93466fa7194d1b88fde34c55
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [14]:
import evaluate
# Load the metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")
bleu_scores = []
rouge_scores = []
meteor_scores = []
exact_match = []
for pair in tqdm(pairs):
    _, reference, hypothesis = pair
    # Skip empty pairs
    if not reference.strip() or not hypothesis.strip():
        continue
    # BLEU
    bleu_score = bleu.compute(predictions=[hypothesis], references=[reference])
    bleu_scores.append(bleu_score["bleu"])
    # ROUGE
    rouge_score = rouge.compute(predictions=[hypothesis], references=[reference])
    rouge_scores.append(rouge_score["rougeL"])
    # METEOR
    meteor_score = meteor.compute(predictions=[hypothesis], references=[reference])
    meteor_scores.append(meteor_score["meteor"])
    # Exact match
    exact_match.append(hypothesis.strip() == reference.strip())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
100%|██████████| 200/200 [00:41<00:00,  4.85it/s]


In [15]:
def safe_mean(lst):
    return np.mean(lst) if lst else float('nan')
print(f"BLEU Score (avg): {safe_mean(bleu_scores)}")
print(f"ROUGE-1 F1 (avg): {safe_mean(rouge_scores)}")
print(f"meteor_scores F1 (avg): {safe_mean(meteor_scores)}")
print(f"Exact Match (avg): {safe_mean(exact_match)}")

BLEU Score (avg): 0.015299767089057906
ROUGE-1 F1 (avg): 0.13442987385050942
meteor_scores F1 (avg): 0.213776936952112
Exact Match (avg): 0.0


In [16]:
filename = "pairs_lora_clean_full.json"
with open(filename, "r", encoding="utf-8") as f:
    pairs = json.load(f)

In [17]:
bleu_scores = []
rouge_scores = []
meteor_scores = []
exact_match = [] # i will be very surprised if this ever goes above 0
for pair in tqdm(pairs):
    _, reference, hypothesis = pair
    if not reference.strip() or not hypothesis.strip():
        continue
    bleu_score = bleu.compute(predictions=[hypothesis], references=[reference])
    bleu_scores.append(bleu_score["bleu"])
    rouge_score = rouge.compute(predictions=[hypothesis], references=[reference])
    rouge_scores.append(rouge_score["rougeL"])
    meteor_score = meteor.compute(predictions=[hypothesis], references=[reference])
    meteor_scores.append(meteor_score["meteor"])
    exact_match.append(hypothesis.strip() == reference.strip())

100%|██████████| 200/200 [00:39<00:00,  5.00it/s]


In [18]:
def safe_mean(lst):
    return np.mean(lst) if lst else float('nan')
print(f"BLEU Score (avg): {safe_mean(bleu_scores)}")
print(f"ROUGE-1 F1 (avg): {safe_mean(rouge_scores)}")
print(f"meteor_scores F1 (avg): {safe_mean(meteor_scores)}")
print(f"Exact Match (avg): {safe_mean(exact_match)}")

BLEU Score (avg): 0.023757218429303343
ROUGE-1 F1 (avg): 0.16138200899900546
meteor_scores F1 (avg): 0.21340778507528463
Exact Match (avg): 0.0
